# Download and convert to Geopackage

This downloads SWOT Pixel Cloud products from hydroweb.next (API-Key necessary) based on a region and a period of interest.
Then is extracts information contained in the area of interest for your study, stores everything in a Geopackage Database for future use.
Geopackage is a convenient data storage format, based on SQL, and is compatible with QGIS.


## Setting the region and period of interest
Using a geopackage layer, preliminary created with, e.g. QGIS, to limit data download and database

In [1]:
from pixcdust.downloaders.hydroweb_next import PixCDownloader
import geopandas as gpd
from datetime import datetime

In [2]:
# reading the area of interest polygon (could have been set)
# gdf_geom = gpd.read_file('/home/hysope2/STUDIES/SWOT_Panama/DATA/aoi.gpkg')
gdf_geom = gpd.read_file("../data/aoi.gpkg")

# Limiting time period
dates = (
    datetime(2024,8,1),
    datetime(2024,8,15),
)


## Download
This will unfortunately lead to downloading many big files (that will be removed later). This is the only way right now, but the hydroweb.next team is working on improving that.

In [3]:
pixcdownloader = PixCDownloader(
    gdf_geom,
    dates,
    verbose=0,
    path_download='/tmp/pixc',
    )
pixcdownloader.search_download()

## Extraction
Now we have all necessary files, let us extract key variables within area of interest in a geopackage database.
This geopackage format is quite efficient (though not the most efficient), and may easily be visualized in, e.g., QGIS
We are using the same geodataframe to limit the data to the area of interest

In [4]:
from pixcdust.converters.gpkg import Nc2GpkgConverter
from glob import glob

In [5]:
pixc = Nc2GpkgConverter(
            glob(pixcdownloader.path_download+'/*/*nc'),
            "/tmp/pixc_gpkg.gpkg",
            variables=['height', 'sig0', 'classification'],
            area_of_interest=gdf_geom,
            mode='o',
        )
pixc.database_from_nc()

100%|████████████████████████████████████████████████████████████████████| 2/2 [00:52<00:00, 26.48s/it]


database has been succesfully created, we can remove the raw files

In [6]:
# import shutil
# shutil.rmtree('/tmp/pixc')

# Read the database
Previous steps are not necessary

Now we can open this database in a GeoDataFrame, load it in, e.g., QGIS, etc.

In [7]:
from pixcdust.readers.gpkg import GpkgReader

# nb: you may specify 
pixc_read = GpkgReader(
    "/tmp/pixc_gpkg.gpkg"
)
pixc_read.read()
pixc_read.data

100%|████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00,  3.38it/s]


,height,sig0,classification,geoid,latitude,longitude,wse,geometry
0,94.072586,5.111407,1.0,49.683472,43.399101,-1.475007,44.389114,POINT (-1.47501 43.39910)
1,91.988503,2.306671,2.0,49.682842,43.399265,-1.474796,42.305660,POINT (-1.47480 43.39926)
2,97.418854,0.778096,1.0,49.682732,43.399272,-1.474838,47.736122,POINT (-1.47484 43.39927)
3,99.751564,0.835120,1.0,49.680717,43.399527,-1.475134,50.070847,POINT (-1.47513 43.39953)
4,89.247116,3.606017,2.0,49.686085,43.398929,-1.474051,39.561031,POINT (-1.47405 43.39893)
...,...,...,...,...,...,...,...,...
530317,89.535172,0.735030,1.0,49.064987,43.564340,-1.394668,40.470184,POINT (-1.39467 43.56434)
530318,94.091469,0.177528,1.0,49.064285,43.564356,-1.395633,45.027184,POINT (-1.39563 43.56436)
530319,90.996582,-0.089914,1.0,49.064289,43.564359,-1.395620,41.932293,POINT (-1.39562 43.56436)
530320,93.612373,0.340582,1.0,49.064327,43.564391,-1.395450,44.548046,POINT (-1.39545 43.56439)


Enjoy!